In [1]:
#Add the directory
%cd /home/ubuntu/Desktop/Class_work/ee502/

/home/ubuntu/Desktop/Class_work/ee502


In [2]:
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
#pd.set_option('max_columns', None)

In [3]:
#loading the dataset
dataset = pd.read_excel("All_data.xlsx")
#Display forst rows
dataset.head()

,Participant,mean_ch1,mean_ch2,mean_ch3,mean_ch4,mean_ch5,mean_ch6,mean_ch7,mean_ch8,Standard Deviation_ch1,...,Shannon_Entropy_ch7,Shannon_Entropy_ch8,Power Spectral Density_ch1,Power Spectral Density_ch2,Power Spectral Density_ch3,Power Spectral Density_ch4,Power Spectral Density_ch5,Power Spectral Density_ch6,Power Spectral Density_ch7,Power Spectral Density_ch8
0,2,-68.082074,-51.031745,-10.527269,2.315528,-0.019401,6.305867,10.851360,3.708743,26.700413,...,-9.517344e+06,-9.517344e+06,4588.751100,1706.750311,51.384037,45.251309,1.758736e-03,30.019523,107.622079,31.173689
1,2,-77.006955,-59.638360,-13.851066,3.301459,-0.035965,7.818291,16.363432,4.544376,29.841539,...,-1.294499e+07,-1.294499e+07,2155.936716,1489.069802,90.075615,33.100494,7.668241e-04,60.920785,176.719171,64.948531
2,2,-25.308444,-22.488517,-5.350912,1.393713,-0.001656,-1.068934,11.045225,-0.096602,94.483683,...,-1.814937e+07,-1.814937e+07,3155.364595,1431.098870,87.904151,61.617285,1.653921e-04,138.225621,295.970483,101.511013
3,2,210.559327,158.651936,33.435422,-14.904149,0.010851,-36.535425,-42.368458,-21.192230,229.042799,...,-2.445025e+08,-2.445025e+08,15975.093578,11641.262386,828.512057,144.906923,1.021009e-06,816.783336,1435.397809,286.066135
4,2,236.851464,180.828302,39.804005,-14.173984,0.008831,-34.439719,-53.176788,-23.603237,214.241661,...,-2.619810e+08,-2.619810e+08,38706.466667,39402.082756,915.369729,250.254071,5.530137e-07,762.465900,1378.583475,279.760111


In [4]:
# distributing the dataset into two components X and Y
X = dataset.iloc[:, 1:89] #Independent variables
y = dataset.iloc[:, 0] #Dependent variable

In [5]:
# Split the data into Training set and Testing set
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) #80% train and 20% test

In [6]:
#perform Z-normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
  
X_train = sc.fit_transform(X_train) #normalize training data
X_test = sc.transform(X_test) #normalize test data

In [7]:
#normalization creates an output's array,convert an array to dataframe
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [8]:
# Create correlation matrix
corr_matrix = X_train.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features with correlation greater than 0.95
X_train.drop(to_drop, axis=1, inplace=True)
X_test.drop(to_drop,axis=1,inplace=True)


#Plot the matrix
#plt.figure(figsize=(20,10))
#sns.heatmap(corr_matrix, cmap=plt.cm.CMRmap_r,annot=True)
#plt.show()  
#print(corr_matrix)

/tmp/ipykernel_379500/2992914116.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [9]:
#as per the correlation matrix the features seems to be correlated so applying PCA 
from sklearn.decomposition import PCA

pca = PCA(0.95) #PCA with 95% variance
X_train = pca.fit_transform(X_train) #PCA on train data
X_test = pca.transform(X_test)#PCA on test data

In [10]:
#explain variance of each PC
pca.explained_variance_ratio_

array([0.20640523, 0.13077815, 0.08778915, 0.05483546, 0.05149624,
       0.0383067 , 0.03275205, 0.02967696, 0.02761208, 0.02608153,
       0.02301093, 0.02177069, 0.01999845, 0.01864783, 0.01823651,
       0.01747691, 0.0170277 , 0.01632081, 0.01564278, 0.0153566 ,
       0.01432517, 0.01365504, 0.0130142 , 0.01238056, 0.01146855,
       0.01079199, 0.0091866 ])

Linear Support Vector Machine Model

In [11]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Create a svm Classifier
model_lin = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
model_lin.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model_lin.predict(X_test)

In [12]:
#Print the classification report
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           2       0.97      0.99      0.98       136
           3       0.97      0.92      0.95       116
           4       0.90      0.87      0.89       129
           6       0.87      0.94      0.90       129
           7       0.95      0.94      0.94       129
           8       0.78      0.78      0.78       124
           9       0.82      0.89      0.85       120
          10       0.66      0.71      0.69       125
          11       0.73      0.66      0.69       129
          12       0.64      0.54      0.59       118
          13       0.66      0.70      0.68       124
          14       0.91      0.93      0.92       138

    accuracy                           0.83      1517
   macro avg       0.82      0.82      0.82      1517
weighted avg       0.83      0.83      0.82      1517



In [13]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision
print("Precision:",metrics.precision_score(y_test, y_pred,average='macro'))

# Model Recall
print("Recall:",metrics.recall_score(y_test, y_pred,average='macro'))

Accuracy: 0.8259723137771918
Precision: 0.8226271843416589
Recall: 0.8229584842003743


SVM hyper parameter tuning with Grid Search

In [14]:
from sklearn.model_selection import GridSearchCV
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) #Grid model definition
  
grid.fit(X_train, y_train) #fit the grid mode

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.085 total time=   0.9s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.086 total time=   0.9s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.085 total time=   0.9s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.085 total time=   0.9s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.085 total time=   0.9s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.773 total time=   0.2s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.792 total time=   0.2s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.794 total time=   0.2s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.777 total time=   0.2s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.781 total time=   0.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.638 total time=   0.7s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear']},
             verbose=3)

In [15]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=100, gamma=0.01)


In [16]:
#Prediction on grid search model
grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           2       0.99      0.99      0.99       136
           3       0.93      0.94      0.94       116
           4       0.94      0.92      0.93       129
           6       0.94      0.95      0.95       129
           7       0.95      0.96      0.95       129
           8       0.90      0.84      0.87       124
           9       0.92      0.93      0.93       120
          10       0.74      0.80      0.77       125
          11       0.88      0.88      0.88       129
          12       0.90      0.83      0.86       118
          13       0.79      0.80      0.80       124
          14       0.95      0.96      0.96       138

    accuracy                           0.90      1517
   macro avg       0.90      0.90      0.90      1517
weighted avg       0.90      0.90      0.90      1517



Support vector machine model with best parameters suggested by grid search

In [17]:
#Model with best hyperparameters suggested by gridsearch
model_hyp = svm.SVC(kernel='rbf', C=100, gamma=0.01)

#Train the model using the training sets
model_hyp.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model_hyp.predict(X_test)

In [18]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision
print("Precision:",metrics.precision_score(y_test, y_pred,average='macro'))

# Model Recall
print("Recall:",metrics.recall_score(y_test, y_pred,average='macro'))

Accuracy: 0.9024390243902439
Precision: 0.9022789132704737
Recall: 0.900882185055348
